In [81]:
import pandas as pd
import numpy as np
from textblob import TextBlob
PATH = 'data/'
df = pd.read_csv(PATH + 'data.csv')
df = df.sample(df.shape[0], random_state=42)
#df.text = df.text_clean.astype(str)
df = df[['id', 'text', 'target']]
print(df.isnull().sum())

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
model = SentimentIntensityAnalyzer()


def sentiment_scores(x):
    score = model.polarity_scores(x.text)
    sent_neg, sent_neu, sent_pos, sent_compound = score['neg'], score['neu'], score['pos'], score['compound']
    blob = TextBlob(x.text)
    blob_pol, blob_subj = blob.sentiment.polarity, blob.sentiment.subjectivity
    return sent_neg, sent_neu, sent_pos, sent_compound, blob_pol, blob_subj

df[['sent_neg', 'sent_neu', 'sent_pos', 'sent_compound', 'blob_pol', 'blob_subj']] = df.apply(sentiment_scores, axis=1, result_type='expand')
#df.drop('text', axis=1)
#df.to_csv(PATH + 'sentiment_features.csv')
print(df.shape)
print(df.dtypes)


id        0   
text      0   
target    3263
dtype: int64


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/pranjal/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


(10876, 9)
id               int64  
text             object 
target           float64
sent_neg         float64
sent_neu         float64
sent_pos         float64
sent_compound    float64
blob_pol         float64
blob_subj        float64
dtype: object


In [72]:
df.corr()

,id,target,sent_neg,sent_neu,sent_pos,sent_compound,blob_pol,blob_subj
id,1.000000,0.060781,0.027833,-0.051201,0.037383,-0.015307,0.020124,0.026477
target,0.060781,1.000000,0.104503,0.053307,-0.210443,-0.231723,-0.106022,-0.080452
sent_neg,0.027833,0.104503,1.000000,-0.736877,-0.240151,-0.773818,-0.296920,0.026113
sent_neu,-0.051201,0.053307,-0.736877,1.000000,-0.479280,0.235792,-0.001375,-0.212321
sent_pos,0.037383,-0.210443,-0.240151,-0.479280,1.000000,0.666042,0.387465,0.271000
sent_compound,-0.015307,-0.231723,-0.773818,0.235792,0.666042,1.000000,0.439911,0.110579
blob_pol,0.020124,-0.106022,-0.296920,-0.001375,0.387465,0.439911,1.000000,0.152180
blob_subj,0.026477,-0.080452,0.026113,-0.212321,0.271000,0.110579,0.152180,1.000000


In [70]:
temp = df[df.target.notna()]
x, y = temp.drop(['target', 'id', 'text'], axis=1), temp['target']
x.fillna('None', inplace=True)
x.isnull().sum()

sent_neg         0
sent_neu         0
sent_pos         0
sent_compound    0
blob_pol         0
blob_subj        0
dtype: int64

In [84]:

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

text_features = []
cat_features = []

from catboost import CatBoostClassifier
model = CatBoostClassifier(depth = 3, eval_metric='Accuracy')
model.fit(x_train, y_train, eval_set=(x_test, y_test), cat_features=cat_features, text_features=text_features)


Learning rate set to 0.047906
0:	learn: 0.6177519	test: 0.6024518	best: 0.6024518 (0)	total: 4.71ms	remaining: 4.7s
1:	learn: 0.6224432	test: 0.6042032	best: 0.6042032 (1)	total: 9.58ms	remaining: 4.78s
2:	learn: 0.6226309	test: 0.6024518	best: 0.6042032 (1)	total: 13.5ms	remaining: 4.5s
3:	learn: 0.6335147	test: 0.6133975	best: 0.6133975 (3)	total: 18ms	remaining: 4.49s
4:	learn: 0.6288234	test: 0.6055166	best: 0.6133975 (3)	total: 21.7ms	remaining: 4.32s
5:	learn: 0.6280728	test: 0.6033275	best: 0.6133975 (3)	total: 28.6ms	remaining: 4.73s
6:	learn: 0.6235691	test: 0.6046410	best: 0.6133975 (3)	total: 33.5ms	remaining: 4.76s
7:	learn: 0.6333271	test: 0.6138354	best: 0.6138354 (7)	total: 37.5ms	remaining: 4.65s
8:	learn: 0.6331394	test: 0.6147110	best: 0.6147110 (8)	total: 41.9ms	remaining: 4.61s
9:	learn: 0.6280728	test: 0.6094571	best: 0.6147110 (8)	total: 45.9ms	remaining: 4.54s
10:	learn: 0.6342653	test: 0.6147110	best: 0.6147110 (8)	total: 50.4ms	remaining: 4.53s
11:	learn: 0.634

102:	learn: 0.6654157	test: 0.6440455	best: 0.6453590 (98)	total: 569ms	remaining: 4.96s
103:	learn: 0.6654157	test: 0.6440455	best: 0.6453590 (98)	total: 575ms	remaining: 4.95s
104:	learn: 0.6652280	test: 0.6440455	best: 0.6453590 (98)	total: 580ms	remaining: 4.95s
105:	learn: 0.6652280	test: 0.6431699	best: 0.6453590 (98)	total: 587ms	remaining: 4.95s
106:	learn: 0.6646650	test: 0.6436077	best: 0.6453590 (98)	total: 594ms	remaining: 4.96s
107:	learn: 0.6648527	test: 0.6436077	best: 0.6453590 (98)	total: 603ms	remaining: 4.98s
108:	learn: 0.6652280	test: 0.6449212	best: 0.6453590 (98)	total: 609ms	remaining: 4.98s
109:	learn: 0.6656033	test: 0.6453590	best: 0.6453590 (98)	total: 616ms	remaining: 4.98s
110:	learn: 0.6657910	test: 0.6453590	best: 0.6453590 (98)	total: 620ms	remaining: 4.97s
111:	learn: 0.6657910	test: 0.6449212	best: 0.6453590 (98)	total: 625ms	remaining: 4.96s
112:	learn: 0.6650403	test: 0.6431699	best: 0.6453590 (98)	total: 630ms	remaining: 4.95s
113:	learn: 0.6652280

215:	learn: 0.6701070	test: 0.6501751	best: 0.6506130 (207)	total: 1.15s	remaining: 4.17s
216:	learn: 0.6697317	test: 0.6514886	best: 0.6514886 (216)	total: 1.16s	remaining: 4.18s
217:	learn: 0.6699193	test: 0.6519264	best: 0.6519264 (217)	total: 1.16s	remaining: 4.17s
218:	learn: 0.6701070	test: 0.6519264	best: 0.6519264 (217)	total: 1.18s	remaining: 4.21s
219:	learn: 0.6701070	test: 0.6519264	best: 0.6519264 (217)	total: 1.2s	remaining: 4.24s
220:	learn: 0.6693564	test: 0.6514886	best: 0.6519264 (217)	total: 1.2s	remaining: 4.23s
221:	learn: 0.6695440	test: 0.6514886	best: 0.6519264 (217)	total: 1.21s	remaining: 4.24s
222:	learn: 0.6697317	test: 0.6497373	best: 0.6519264 (217)	total: 1.22s	remaining: 4.24s
223:	learn: 0.6702946	test: 0.6497373	best: 0.6519264 (217)	total: 1.22s	remaining: 4.24s
224:	learn: 0.6699193	test: 0.6497373	best: 0.6519264 (217)	total: 1.23s	remaining: 4.23s
225:	learn: 0.6701070	test: 0.6497373	best: 0.6519264 (217)	total: 1.24s	remaining: 4.26s
226:	learn: 

321:	learn: 0.6732971	test: 0.6519264	best: 0.6528021 (273)	total: 1.92s	remaining: 4.04s
322:	learn: 0.6732971	test: 0.6519264	best: 0.6528021 (273)	total: 1.93s	remaining: 4.04s
323:	learn: 0.6734847	test: 0.6523643	best: 0.6528021 (273)	total: 1.93s	remaining: 4.03s
324:	learn: 0.6734847	test: 0.6523643	best: 0.6528021 (273)	total: 1.94s	remaining: 4.02s
325:	learn: 0.6740477	test: 0.6514886	best: 0.6528021 (273)	total: 1.94s	remaining: 4.02s
326:	learn: 0.6734847	test: 0.6514886	best: 0.6528021 (273)	total: 1.95s	remaining: 4.01s
327:	learn: 0.6738600	test: 0.6510508	best: 0.6528021 (273)	total: 1.96s	remaining: 4.01s
328:	learn: 0.6742353	test: 0.6523643	best: 0.6528021 (273)	total: 1.96s	remaining: 4s
329:	learn: 0.6736724	test: 0.6528021	best: 0.6528021 (273)	total: 1.97s	remaining: 4.01s
330:	learn: 0.6753612	test: 0.6541156	best: 0.6541156 (330)	total: 1.98s	remaining: 4s
331:	learn: 0.6755489	test: 0.6541156	best: 0.6541156 (330)	total: 1.98s	remaining: 3.99s
332:	learn: 0.67

440:	learn: 0.6804278	test: 0.6549912	best: 0.6567426 (428)	total: 2.69s	remaining: 3.41s
441:	learn: 0.6802402	test: 0.6554291	best: 0.6567426 (428)	total: 2.7s	remaining: 3.41s
442:	learn: 0.6806155	test: 0.6554291	best: 0.6567426 (428)	total: 2.71s	remaining: 3.4s
443:	learn: 0.6806155	test: 0.6554291	best: 0.6567426 (428)	total: 2.71s	remaining: 3.4s
444:	learn: 0.6809908	test: 0.6554291	best: 0.6567426 (428)	total: 2.72s	remaining: 3.4s
445:	learn: 0.6811785	test: 0.6549912	best: 0.6567426 (428)	total: 2.73s	remaining: 3.4s
446:	learn: 0.6811785	test: 0.6549912	best: 0.6567426 (428)	total: 2.74s	remaining: 3.39s
447:	learn: 0.6811785	test: 0.6549912	best: 0.6567426 (428)	total: 2.75s	remaining: 3.38s
448:	learn: 0.6811785	test: 0.6549912	best: 0.6567426 (428)	total: 2.75s	remaining: 3.38s
449:	learn: 0.6813661	test: 0.6549912	best: 0.6567426 (428)	total: 2.76s	remaining: 3.37s
450:	learn: 0.6813661	test: 0.6558669	best: 0.6567426 (428)	total: 2.77s	remaining: 3.37s
451:	learn: 0.6

532:	learn: 0.6849315	test: 0.6593695	best: 0.6611208 (518)	total: 3.27s	remaining: 2.86s
533:	learn: 0.6849315	test: 0.6593695	best: 0.6611208 (518)	total: 3.27s	remaining: 2.86s
534:	learn: 0.6849315	test: 0.6593695	best: 0.6611208 (518)	total: 3.28s	remaining: 2.85s
535:	learn: 0.6849315	test: 0.6593695	best: 0.6611208 (518)	total: 3.29s	remaining: 2.85s
536:	learn: 0.6849315	test: 0.6593695	best: 0.6611208 (518)	total: 3.3s	remaining: 2.85s
537:	learn: 0.6849315	test: 0.6593695	best: 0.6611208 (518)	total: 3.31s	remaining: 2.84s
538:	learn: 0.6854945	test: 0.6589317	best: 0.6611208 (518)	total: 3.32s	remaining: 2.84s
539:	learn: 0.6854945	test: 0.6589317	best: 0.6611208 (518)	total: 3.32s	remaining: 2.83s
540:	learn: 0.6854945	test: 0.6589317	best: 0.6611208 (518)	total: 3.33s	remaining: 2.83s
541:	learn: 0.6851192	test: 0.6589317	best: 0.6611208 (518)	total: 3.33s	remaining: 2.82s
542:	learn: 0.6853068	test: 0.6584939	best: 0.6611208 (518)	total: 3.34s	remaining: 2.81s
543:	learn:

624:	learn: 0.6881216	test: 0.6598074	best: 0.6615587 (578)	total: 3.85s	remaining: 2.31s
625:	learn: 0.6881216	test: 0.6598074	best: 0.6615587 (578)	total: 3.85s	remaining: 2.3s
626:	learn: 0.6875586	test: 0.6606830	best: 0.6615587 (578)	total: 3.86s	remaining: 2.3s
627:	learn: 0.6875586	test: 0.6611208	best: 0.6615587 (578)	total: 3.88s	remaining: 2.29s
628:	learn: 0.6881216	test: 0.6619965	best: 0.6619965 (628)	total: 3.88s	remaining: 2.29s
629:	learn: 0.6881216	test: 0.6619965	best: 0.6619965 (628)	total: 3.89s	remaining: 2.29s
630:	learn: 0.6877463	test: 0.6619965	best: 0.6619965 (628)	total: 3.9s	remaining: 2.28s
631:	learn: 0.6877463	test: 0.6619965	best: 0.6619965 (628)	total: 3.9s	remaining: 2.27s
632:	learn: 0.6877463	test: 0.6619965	best: 0.6619965 (628)	total: 3.91s	remaining: 2.27s
633:	learn: 0.6881216	test: 0.6619965	best: 0.6619965 (628)	total: 3.91s	remaining: 2.26s
634:	learn: 0.6881216	test: 0.6619965	best: 0.6619965 (628)	total: 3.92s	remaining: 2.25s
635:	learn: 0.

725:	learn: 0.6918746	test: 0.6615587	best: 0.6633100 (670)	total: 4.42s	remaining: 1.67s
726:	learn: 0.6918746	test: 0.6615587	best: 0.6633100 (670)	total: 4.42s	remaining: 1.66s
727:	learn: 0.6922500	test: 0.6615587	best: 0.6633100 (670)	total: 4.43s	remaining: 1.65s
728:	learn: 0.6918746	test: 0.6619965	best: 0.6633100 (670)	total: 4.44s	remaining: 1.65s
729:	learn: 0.6922500	test: 0.6619965	best: 0.6633100 (670)	total: 4.45s	remaining: 1.65s
730:	learn: 0.6922500	test: 0.6619965	best: 0.6633100 (670)	total: 4.45s	remaining: 1.64s
731:	learn: 0.6922500	test: 0.6619965	best: 0.6633100 (670)	total: 4.46s	remaining: 1.63s
732:	learn: 0.6920623	test: 0.6619965	best: 0.6633100 (670)	total: 4.46s	remaining: 1.63s
733:	learn: 0.6924376	test: 0.6619965	best: 0.6633100 (670)	total: 4.47s	remaining: 1.62s
734:	learn: 0.6924376	test: 0.6619965	best: 0.6633100 (670)	total: 4.47s	remaining: 1.61s
735:	learn: 0.6924376	test: 0.6615587	best: 0.6633100 (670)	total: 4.48s	remaining: 1.61s
736:	learn

827:	learn: 0.6976919	test: 0.6628722	best: 0.6633100 (670)	total: 5s	remaining: 1.04s
828:	learn: 0.6978795	test: 0.6628722	best: 0.6633100 (670)	total: 5.01s	remaining: 1.03s
829:	learn: 0.6984425	test: 0.6624343	best: 0.6633100 (670)	total: 5.04s	remaining: 1.03s
830:	learn: 0.6986301	test: 0.6628722	best: 0.6633100 (670)	total: 5.05s	remaining: 1.03s
831:	learn: 0.6986301	test: 0.6628722	best: 0.6633100 (670)	total: 5.06s	remaining: 1.02s
832:	learn: 0.6986301	test: 0.6628722	best: 0.6633100 (670)	total: 5.07s	remaining: 1.02s
833:	learn: 0.6986301	test: 0.6628722	best: 0.6633100 (670)	total: 5.07s	remaining: 1.01s
834:	learn: 0.6982548	test: 0.6619965	best: 0.6633100 (670)	total: 5.08s	remaining: 1s
835:	learn: 0.6982548	test: 0.6619965	best: 0.6633100 (670)	total: 5.08s	remaining: 997ms
836:	learn: 0.6982548	test: 0.6619965	best: 0.6633100 (670)	total: 5.09s	remaining: 991ms
837:	learn: 0.6980672	test: 0.6619965	best: 0.6633100 (670)	total: 5.09s	remaining: 984ms
838:	learn: 0.69

933:	learn: 0.7006943	test: 0.6633100	best: 0.6637478 (928)	total: 5.56s	remaining: 393ms
934:	learn: 0.7006943	test: 0.6633100	best: 0.6637478 (928)	total: 5.57s	remaining: 387ms
935:	learn: 0.7006943	test: 0.6633100	best: 0.6637478 (928)	total: 5.58s	remaining: 381ms
936:	learn: 0.7006943	test: 0.6633100	best: 0.6637478 (928)	total: 5.58s	remaining: 375ms
937:	learn: 0.7006943	test: 0.6633100	best: 0.6637478 (928)	total: 5.59s	remaining: 369ms
938:	learn: 0.7010696	test: 0.6628722	best: 0.6637478 (928)	total: 5.59s	remaining: 364ms
939:	learn: 0.7008820	test: 0.6628722	best: 0.6637478 (928)	total: 5.6s	remaining: 357ms
940:	learn: 0.7010696	test: 0.6628722	best: 0.6637478 (928)	total: 5.6s	remaining: 351ms
941:	learn: 0.7012573	test: 0.6628722	best: 0.6637478 (928)	total: 5.61s	remaining: 345ms
942:	learn: 0.7008820	test: 0.6633100	best: 0.6637478 (928)	total: 5.61s	remaining: 339ms
943:	learn: 0.7008820	test: 0.6633100	best: 0.6637478 (928)	total: 5.62s	remaining: 333ms
944:	learn: 